In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd



/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[    0, 32099,    10,    27,   333,    25,    71,    10,   333,    71,
            10,   280,    19,  2530,   705,    71,    10,   280,    19,  2530]])
Generated: : I love you A: love A: life is meaning less A: life is meaning


In [2]:
class QuoteInterpreter():
  '''
  init - should accept model name
  set model and tokenizer

  prompt func (df - quote, interpretation, technique) -> df with interpretation

  df - quote, interpretation
  technique - "zero shot", "few shot", cot

  for each quote in df:
    prompt = get_prompt()

    model(prompt)
    decode()
    append to df

  get_prompt(quote_idx, df, technique):
    if zero_shot:
      prompt(quote) --- zero shot prompt
    if few_shot:
      prompt(quote, df) --- few shot prompt
      get_similar quote_interpretation(quote, tag)
      few_shot_prompt(quote, similar_interpretation)
    else:
      cot - get_cot_example(quote, tag)
      cot_prompt(quote, cot_example
  '''

  def __init__(self, model_name) -> None:
    self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
    self.tokenizer = AutoTokenizer.from_pretrained(model_name)

  def prompt(self, data, method="zero_shot"):
    for i in range(len(data)):
      prompt = self.get_prompt(i, data, method)
      input_ids = self.tokenizer(prompt, return_tensors="pt").input_ids
      output_ids = self.model.generate(input_ids)
      data.loc[i, "interpretation"] = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return data

  def get_prompt(self, quote_idx, data, method):

    if method == "cot":
      return self.cot_prompt(data.loc[quote_idx, "quote"], data.loc[quote_idx, "tags"])

    if method == "few_shot":
      return self.few_shot_prompt(data.loc[quote_idx, "quote"], data.loc[quote_idx, "tag"])

    return self.zero_shot_prompt(data.loc[quote_idx, "quote"])

  def zero_shot_prompt(self, quote):
    #TODO - add the correct prompt for zero-shot
    return f"Interpret the meaning of the quote in 1-2 sentences Q: {quote}\n A: "

  def few_shot_prompt(self, quote, df):
    #TODO - add the logic to select the demonstrations
    return f"Interpret the meaning of the quote in 1-2 sentences Q: {quote}\n A: "



In [8]:
data = {
    "quote": ["Love is the greatest gift",
              "Life is too short to be anything but happy",
              "The best way to find yourself is to lose yourself in the service of others",
              "A friend is someone who knows all about you and still loves you",
              "The greatest glory in living lies not in never falling, but in rising every time we fall",
              "The only person you are destined to become is the person you decide to be",
              "It's not how much we give, but how much love we put into giving",
              "Life is what happens when you're busy making other plans",
              "The greatest wealth is to live content with little",
              "You only live once, but if you do it right, once is enough"],
    "tag": ["love", "life", "love", "love", "life", "life", "love", "life", "life", "life"],
}

df = pd.DataFrame(data)

model_name = "t5-3b"

interpreter = QuoteInterpreter(model_name)



/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


                                               quote   tag  \
0                          Love is the greatest gift  love   
1         Life is too short to be anything but happy  life   
2  The best way to find yourself is to lose yours...  love   
3  A friend is someone who knows all about you an...  love   
4  The greatest glory in living lies not in never...  life   
5  The only person you are destined to become is ...  life   
6  It's not how much we give, but how much love w...  love   
7  Life is what happens when you're busy making o...  life   
8  The greatest wealth is to live content with li...  life   
9  You only live once, but if you do it right, on...  life   

                                      interpretation  
0  Q: Love is the greatest gift A: Love is the gr...  
1   Q: Life is too short to be anything but happy A:  
2  Q: The best way to find yourself is to lose yo...  
3  A friend is someone who knows all about you an...  
4  The greatest glory in living lies not i

In [9]:
df = interpreter.prompt(df, method="zero_shot")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


                                               quote   tag  \
0                          Love is the greatest gift  love   
1         Life is too short to be anything but happy  life   
2  The best way to find yourself is to lose yours...  love   
3  A friend is someone who knows all about you an...  love   
4  The greatest glory in living lies not in never...  life   
5  The only person you are destined to become is ...  life   
6  It's not how much we give, but how much love w...  love   
7  Life is what happens when you're busy making o...  life   
8  The greatest wealth is to live content with li...  life   
9  You only live once, but if you do it right, on...  life   

                                      interpretation  
0  Q: Love is the greatest gift A: Love is the gr...  
1   Q: Life is too short to be anything but happy A:  
2  Q: The best way to find yourself is to lose yo...  
3  A friend is someone who knows all about you an...  
4  The greatest glory in living lies not i

In [10]:
df

,quote,tag,interpretation
0,Love is the greatest gift,love,Q: Love is the greatest gift A: Love is the gr...
1,Life is too short to be anything but happy,life,Q: Life is too short to be anything but happy A:
2,The best way to find yourself is to lose yours...,love,Q: The best way to find yourself is to lose yo...
3,A friend is someone who knows all about you an...,love,A friend is someone who knows all about you an...
4,The greatest glory in living lies not in never...,life,The greatest glory in living lies not in never...
5,The only person you are destined to become is ...,life,.: a person a....
6,"It's not how much we give, but how much love w...",love,"Q: It's not how much we give, but how much lov..."
7,Life is what happens when you're busy making o...,life,Q: Life is what happens when you're busy makin...
8,The greatest wealth is to live content with li...,life,Q: The greatest wealth is to live content with...
9,"You only live once, but if you do it right, on...",life,.:::.....


from matplotlib import pyplot as plt
import seaborn as sns
df.groupby('tag').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)